In [ ]:
# !pip install evaluate
# !pip install datasets
# !pip install fsspec==2023.9.2
# from datasets import Dataset, load_dataset,  DatasetDict
# !pip install bitsandbytes
# !pip install accelerate
#from tqdm.notebook import tqdm

In [ ]:
from transformers import pipeline
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
from torch import Tensor
from tqdm import tqdm


import os
from tqdm import tqdm
import torch

import torch
import pandas as pd
import numpy as np

import json
from typing import List, Dict

from huggingface_hub import login
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


import gc


with open("HF_TOKEN.txt", "r") as f:
    token = f.read().strip()

login(token=token)

import time

In [ ]:
ruta = './data/Silver/Cleaned_data_LOPD.parquet' 

df = pd.read_parquet(ruta)

df

In [ ]:
df.columns

In [ ]:
dataset = Dataset.from_pandas(df)
dataset

In [ ]:
### 1.  Analizamos el sentimiento de todos los tweets

dataset = Dataset.from_pandas(df)

model_name = "MorcuendeA/Joan_Roig_twitter_sentiment"
modelo_clasificacion = pipeline("text-classification", model=model_name, tokenizer=model_name)


label2id = {"negativo": 0, "positivo": 1, "neutral":2}
documentos = [x["tweets"] for x in dataset]


batch_size = 512
preds = []

for i in tqdm(range(0, len(documentos), batch_size)):
    batch = documentos[i:i+batch_size]
    with torch.no_grad():
        batch_preds = modelo_clasificacion(batch)
    preds.extend(batch_preds)


num_preds = [label2id[x["label"]] for x in preds]

df['sentimiento'] = num_preds
df["sentimiento"] = df["sentimiento"].map({1: "positivo", 0: "negativo", 2:"neutral"})

In [ ]:
print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
### limpieza de la memoria GPU
print("========================")
del  preds, batch_preds, modelo_clasificacion
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

In [ ]:
### 2. Detectamos si los mensajes hablan sobre la marca mercadona.
#seleccionamos solamente los de mercadona
df_intermedio = df[df['search']=='mercadona']

dataset_intermedio = Dataset.from_pandas(df_intermedio)


model_name = "MorcuendeA/mercadona_Brand_reference_detector"
modelo_clasificacion = pipeline("text-classification", model=model_name, tokenizer='Twitter/twhin-bert-large')


label2id = {"NO":0, "SI":1}
documentos = [x["tweets"] for x in dataset_intermedio]


batch_size = 512
preds = []

for i in tqdm(range(0, len(documentos), batch_size)):
    batch = documentos[i:i+batch_size]
    with torch.no_grad():
        batch_preds = modelo_clasificacion(batch)
    preds.extend(batch_preds)


num_preds = [label2id[x["label"]] for x in preds]

df_intermedio['imagen_marca'] = num_preds
df_intermedio["imagen_marca"] = df_intermedio["imagen_marca"].map({1: "SI", 0: "NO"})

In [ ]:
# Unir por índice
df = df.join(df_intermedio['imagen_marca'], how="left")

# Rellenar los NaN con "no aplica"
df = df.fillna("No aplica")

In [ ]:
print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
### limpieza de la memoria GPU
print("========================")
del  preds, batch_preds, modelo_clasificacion, dataset_intermedio
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

In [ ]:
### 3  Detectamos si los sentimimientos del mensaje se asocian al producto. y filtramos todos los mensajes para analizarlos.
df_productos = df.copy()



model_name = "MorcuendeA/mercadona_sentiment_product_detection"
modelo_clasificacion = pipeline("text-classification", model=model_name, tokenizer=model_name)
label2id = {"NO":0, "SI":1}


documentos = [x["tweets"] for x in dataset]


batch_size = 512
preds = []

for i in tqdm(range(0, len(documentos), batch_size)):
    batch = documentos[i:i+batch_size]
    with torch.no_grad():
        batch_preds = modelo_clasificacion(batch)
    preds.extend(batch_preds)


num_preds = [label2id[x["label"]] for x in preds]


df_productos['sentimiento_producto'] = num_preds
df_productos["sentimiento_producto"] = df_productos["sentimiento_producto"].map({1: "SI", 0: "NO"})
df_productos = df_productos[df_productos['sentimiento_producto'] == 'SI']
#creamos dataset para el siguiente modelo. 
dataset_productos = Dataset.from_pandas(df_productos)

In [ ]:

print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
### limpieza de la memoria GPU
print("========================")
del  preds, batch_preds, modelo_clasificacion
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

In [ ]:
# 4. Detectamos si se están comparando algun producto para extraer el sentimiento referente al prodcuto con T5Gemma.


model_name = "MorcuendeA/product_detection_comparison"



modelo_clasificacion = pipeline("text-classification", model=model_name, tokenizer='microsoft/mdeberta-v3-base')
label2id = {"NO":0, "SI":1}


documentos = [x["tweets"] for x in dataset_productos]


batch_size = 512
preds = []

for i in tqdm(range(0, len(documentos), batch_size)):
    batch = documentos[i:i+batch_size]
    with torch.no_grad():
        batch_preds = modelo_clasificacion(batch)
    preds.extend(batch_preds)


num_preds = [label2id[x["label"]] for x in preds]


df_productos['comparativa_producto'] = num_preds
df_productos["comparativa_producto"] = df_productos["comparativa_producto"].map({1: "SI", 0: "NO"})

In [ ]:
# Unir por índice
df = df.join(df_productos[['sentimiento_producto', 'comparativa_producto']], how="left")

# Rellenar los NaN con "no aplica"
df = df.fillna("No aplica")

In [ ]:

print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
### limpieza de la memoria GPU
print("========================")
del  preds, batch_preds, modelo_clasificacion, dataset_productos, dataset
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

In [ ]:

# Moldeo T5Gemma

model_name = "google/t5gemma-9b-9b-ul2-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)


# al tener dos GPU de 12 G tengo que dividir manualmente el modelo porque el "auto" lo fragmenta 
# y a la hora de generar el output genera error

device_map = {
    'model.encoder': 'cuda:0',
    'model.decoder': 'cuda:1',
    'lm_head': 'cuda:1',
    'model.shared': 'cuda:0'
}


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map= device_map, ## Normalmente "auto" o cuda:0, cuda:1
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    #offload_folder="offload",  # carpeta temporal en disco
)

In [ ]:
df= pd.to_parquet('./data/Silver/Cleaned_data_features.parquet', index=False)

In [ ]:
df= pd.read_parquet('./data/Silver/Cleaned_data_features.parquet')
df

In [ ]:
###### extraer productos.

def create_product_prompt(documento):
    return [{
        "role": "user",
        "content": (
            "Eres un analista experto en mensajes breves. Tu única tarea es extraer información explícita del mensaje.\n"
            "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
            "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
            "Utiliza el formato específico de la pregunta (ej. 'SI'/'NO', 'positivo'/'negativo'/'neutral').\n"
            "Responde siempre en español.\n"
            "Esfuerzate al 100%.\n"
            "Los resultados son muy importantes para mi.\n"
            f"""Tarea: Extrae del texto productos o categorías de supermercado.
                Reglas y pasos a seguir:

                1. **Regla prioritaria antes de extraer**:
                  - Elimina de la consideración toda mención a nombres de tiendas, cadenas o marcas si no vienen acompañadas de un producto específico.
                  - Ejemplos: "Mercadona", "Hacendado" no se consideran productos.
                  - Solo si aparece un producto junto a la marca se incluye (por ejemplo: "galletas Hacendado").

                2. Lee el texto de entrada con atención.

                3. Identifica el producto de mercadona o hacendado:
                  - Incluye artículos específicos como "leche", "detergente", "pan integral".
                  - Incluye categorías generales como "skincare", "cuidado de la piel", "maquillaje", "cosmética", "higiene personal", "productos de limpieza", o equivalentes.
                  - Considera sinónimos y términos en otros idiomas que signifiquen lo mismo.
                  - Considera sabores, colores y adjetivos inseparables como parte del producto, por ejemplo: "jamón serrano", "patatas sabor jamón", "donuts de chocolate".
                  - No incluyas adjetivos sueltos que no sean parte inseparable del producto.

                4. No fragmentar productos compuestos:
                  - Si un producto está formado por varios elementos que aparecen juntos y funcionan como un solo producto, listarlo como una única entrada. Ejemplo: "conos de bacon y queso" → solo un producto.

                5. Excluir elementos que no sean productos o categorías reales:
                  - No incluir marcas, tiendas o cadenas sin producto específico.
                  - No incluir frases como "de Mercadona" o "de Hacendado".
                  - No incluir menciones de productos usadas solo en sentido figurado.
                  - No incluir adjetivos, descriptores o calificativos que no formen parte inseparable del nombre del producto.

                6. Mantener el nombre exacto tal como aparece en el texto:
                  - No corregir, traducir o modificar. 
                      **EXCEPCTO**: Corrige los diminutivos. ej: vinitos a vino 
                  - Respetar mayúsculas, minúsculas y ortografía original.

                7. Eliminar duplicados y genéricos si existen versiones más específicas:
                  - Si un producto está completamente contenido en otro más largo, mantener solo el más específico.
                  - Ejemplo: si aparecen "tinte" y "tinte pelirrojo", incluir solo "tinte pelirrojo".

                8. Formato de respuesta:
                  - Si no hay productos. Responde exactamente: No aplica
                  - Si hay uno o más productos. listarlos en formato:
                    1: Producto_1
                    2: Producto_2
                    3: Producto_3
                    (manteniendo el orden en el que aparecen en el texto).

                9. Paso final de control:
                  - Confirmar que:
                    - No hay marcas o tiendas solas.
                    - No hay duplicados.
                    - No hay versiones genéricas si hay una más específica.

                Ejemplo:
                Texto: 'MADRE MIA, QUE BUENA ESTA LA NOCILLA DEL MERCADONA'
                Respuesta:
                1: Nocilla

                -Ahora analiza:\n"""
            f"Texto: {documento}"
        )
    }]

def process_product_batch(batch_texts, batch_size=4):
    """Procesa un batch de textos para extracción de productos"""
    # Crear prompts para todo el batch
    batch_prompts = [create_product_prompt(texto) for texto in batch_texts]
    
    # Tokenizar todo el batch
    batch_inputs = []
    for prompt in batch_prompts:
        tokenized = tokenizer.apply_chat_template(
            prompt, 
            return_tensors="pt", 
            return_dict=True, 
            add_generation_prompt=True,
            padding=False
        )
        batch_inputs.append(tokenized)
    
    # Extraer input_ids y attention_masks
    input_ids_list = [inputs['input_ids'].squeeze(0) for inputs in batch_inputs]
    attention_mask_list = [inputs['attention_mask'].squeeze(0) for inputs in batch_inputs]
    
    # Hacer padding manual
    max_length = max(len(ids) for ids in input_ids_list)
    
    padded_input_ids = []
    padded_attention_masks = []
    
    for ids, mask in zip(input_ids_list, attention_mask_list):
        padding_length = max_length - len(ids)
        if padding_length > 0:
            # Padding a la izquierda para modelos generativos
            padded_ids = torch.cat([
                torch.full((padding_length,), tokenizer.pad_token_id, dtype=ids.dtype),
                ids
            ])
            padded_mask = torch.cat([
                torch.zeros(padding_length, dtype=mask.dtype),
                mask
            ])
        else:
            padded_ids = ids
            padded_mask = mask
            
        padded_input_ids.append(padded_ids)
        padded_attention_masks.append(padded_mask)
    
    # Convertir a tensores
    batch_input_ids = torch.stack(padded_input_ids).to('cuda:0')
    batch_attention_masks = torch.stack(padded_attention_masks).to('cuda:0')
    
    # Generar respuestas
    with torch.no_grad():
        outputs = model.generate(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_masks,
            max_new_tokens=100,  
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=False,
        )
    
    # Limpiar memoria
    del batch_input_ids, batch_attention_masks
    torch.cuda.empty_cache()
    
    # Decodificar respuestas
    responses = []
    for output in outputs:
        response = tokenizer.decode(output, skip_special_tokens=True)
        responses.append(response.strip())
        
    del outputs
    torch.cuda.empty_cache()
    
    return responses

# Procesamiento principal
producto = []
batch_size = 16  #

# Obtener solo las filas donde sentimiento_producto == 'SI'
rows_to_process = df[df['sentimiento_producto'] == 'SI']
tweets_to_process = rows_to_process["tweets"].tolist()
indices_to_process = rows_to_process.index.tolist()

print(f"Procesando {len(tweets_to_process)} tweets con sentimiento_producto == 'SI'")

processed_results = []
for i in tqdm(range(0, len(tweets_to_process), batch_size), desc="Processing Product Batches"):
    batch_texts = tweets_to_process[i:i+batch_size]
    batch_responses = process_product_batch(batch_texts, batch_size)
    processed_results.extend(batch_responses)

# Crear el resultado final manteniendo el orden original del DataFrame
resultado_map = dict(zip(indices_to_process, processed_results))

# Llenar la lista producto con los resultados correspondientes
for index in df.index:
    if df.loc[index, 'sentimiento_producto'] == 'SI':
        producto.append(resultado_map[index])
    else:
        producto.append('No aplica')

print(f"Procesados {len(producto)} tweets en total")
print(f"Extracciones realizadas: {len(processed_results)}")
print(f"'No aplica' asignados: {len(producto) - len(processed_results)}")

# Agregar la columna al DataFrame
df['producto'] = producto

In [ ]:
df.producto.value_counts()

In [ ]:
fin = time.time()

print(f"Tiempo de ejecución: {fin - inicio:.4f} segundos")

In [ ]:
#limpiar un poco.

In [ ]:
df

In [ ]:
df.to_parquet('Cleaned_data_features_productos_1.parquet' ,index=False) #INTERMEDIO Por si hay error en procesado

In [ ]:
#### SENTIMIENTO DE PRODUCTO MERCADONA O HACENADO FRENTE A COMPETIDOR
def create_product_prompt(documento):
    return [{
      "role": "user",
      "content": (
          "Eres un experto en mensajes breves. Tu única tarea es extraer la información solicitada del mensaje.\n"
          "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
          "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
          "Utiliza el formato específico.\n"
          "Esfuerzate al 100%.\n"
          "Los resultados son muy importantes para mi.\n"
          f""" valora el producto de mercadona o hacendado respecto a producto que se compara.
              - Instrucciones:
                  1. Determina cuál es el producto de la marca mercadona o hacendado.
                  2. valora el producto de mercadona o hacendado respecto a su competidor.
                      - Respuestas posibles: positivo, negativo o neutral.
                  3. Formato de respuesta:
                      - Producto de mercadona o hacendado : valoracion del producto respecto a la competencia. ej: yogur de stracciatella : positivo

          Texto: '{documento}'
          Respuesta:"""
          )
    }]

def process_comparativa_batch(batch_texts, batch_size=4):
    """Procesa un batch de textos para extracción de productos"""
    # Crear prompts para todo el batch
    batch_prompts = [create_product_prompt(texto) for texto in batch_texts]
    
    # Tokenizar todo el batch
    batch_inputs = []
    for prompt in batch_prompts:
        tokenized = tokenizer.apply_chat_template(
            prompt, 
            return_tensors="pt", 
            return_dict=True, 
            add_generation_prompt=True,
            padding=False
        )
        batch_inputs.append(tokenized)
    
    # Extraer input_ids y attention_masks
    input_ids_list = [inputs['input_ids'].squeeze(0) for inputs in batch_inputs]
    attention_mask_list = [inputs['attention_mask'].squeeze(0) for inputs in batch_inputs]
    
    # Hacer padding manual
    max_length = max(len(ids) for ids in input_ids_list)
    
    padded_input_ids = []
    padded_attention_masks = []
    
    for ids, mask in zip(input_ids_list, attention_mask_list):
        padding_length = max_length - len(ids)
        if padding_length > 0:
            # Padding a la izquierda para modelos generativos
            padded_ids = torch.cat([
                torch.full((padding_length,), tokenizer.pad_token_id, dtype=ids.dtype),
                ids
            ])
            padded_mask = torch.cat([
                torch.zeros(padding_length, dtype=mask.dtype),
                mask
            ])
        else:
            padded_ids = ids
            padded_mask = mask
            
        padded_input_ids.append(padded_ids)
        padded_attention_masks.append(padded_mask)
    
    # Convertir a tensores
    batch_input_ids = torch.stack(padded_input_ids).to('cuda:0')
    batch_attention_masks = torch.stack(padded_attention_masks).to('cuda:0')
    
    # Generar respuestas
    with torch.no_grad():
        outputs = model.generate(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_masks,
            max_new_tokens=50,  
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=False,
        )
    
    # Limpiar memoria
    del batch_input_ids, batch_attention_masks
    torch.cuda.empty_cache()
    
    # Decodificar respuestas
    responses = []
    for output in outputs:
        response = tokenizer.decode(output, skip_special_tokens=True)
        responses.append(response.strip())
        
    del outputs
    torch.cuda.empty_cache()
    
    return responses


batch_size = 32  
# Filtrar solo comparativa
rows_to_process = df[df['comparativa_producto'] == 'SI']
tweets_to_process = rows_to_process["tweets"].tolist()
indices_to_process = rows_to_process.index.tolist()

print(f"Procesando {len(tweets_to_process)} tweets con comparativa_producto == 'SI'")

processed_results = []
for i in tqdm(range(0, len(tweets_to_process), batch_size), desc="Processing Comparative Batches"):
    batch_texts = tweets_to_process[i:i+batch_size]
    batch_responses = process_comparativa_batch(batch_texts, batch_size)
    processed_results.extend(batch_responses)

# Mapear resultados al índice correspondiente
resultado_map = dict(zip(indices_to_process, processed_results))

# Actualizar solo filas con comparativa
for index, response in resultado_map.items():
    try:
        prod, sent = response.split(":")
        df.at[index, "producto"] = f"1: {prod.strip()}"
        df.at[index, "sentimiento"] = sent.strip()
    except Exception:
        df.at[index, "producto"] = response
        df.at[index, "sentimiento"] = "error"

print(f"Comparativa: procesadas {len(resultado_map)} filas")


In [ ]:
df.to_parquet('./data/Silver/Cleaned_data_features_productos_2.parquet' ,index=False) #INTERMEDIO

In [ ]:
#vemos la salida anterior y la nueva
df_productos = pd.read_parquet('./data/Silver/Cleaned_data_features_productos_1.parquet') # intermedio
#hay que limpiar las malas salidas del modelo.


df_productos.sentimiento.value_counts()

In [ ]:
df = pd.read_parquet('./data/Silver/Cleaned_data_features_productos_2.parquet') #intermedio

In [ ]:
### hay que arreglar algunas salidas no experadas. 
#error pasa porque hay lista de productos muy larga. 
df.sentimiento.value_counts()

In [ ]:
# df.producto[df.sentimiento == 'negative'].value_counts()

In [ ]:
mapeo = {
    "1: tea": "1: té",
    "1: Watermelon": "1: sandía",
    "1: oil": "1: aceite",
    "1: tea negro": "1: té negro",
    "1: tea de campana": "1: té de campana",
    "1: Oil": "1: aceite",
    "1: oil with shine": "1: aceite con brillo",
    "1: Beer": "1: cerveza",
    "1: Soap": "1: jabón",
    "1: tea bags" : "1: Bolsas de té" 
}

df["producto"] = df["producto"].replace(mapeo)
df.producto[df.sentimiento == 'negative'].value_counts()
#algunos productos en la siguiente fase se arreglaran. 

In [ ]:
mapeo_sentimiento = {
    "negative": "negativo",
    "positive": "positivo",
    "Neutral": "neutral",
    "rebelde": "neutral"
}
df["sentimiento"] = df["sentimiento"].replace(mapeo_sentimiento)


In [ ]:
def restaurar_sentimiento_y_producto(df, df_productos):
    # valores sucios a detectar
    valores_erroneos = [
        "error",
        "valoracion del producto respecto a la competencia.",
        "긍정적", "중립", "부정적", "음성",
        "No se puede determinar la valoración del producto.",
        "no se puede determinar la valoración del producto.",
        "no se puede determinar",
        "No se puede determinar.",
        "No se puede determinar la valoración del producto de mercadona o hacendado.",
        "No es posible determinar la valoración del producto.",
        "no se puede determinar la valoración del producto de mercadona o hacendado.",
        "No se puede determinar la valoración del producto de Mercadona o Hacendado.",
        "No es un producto de mercadona o hacendado.",
        "no se puede determinar la valoración del producto",
        "no se puede determinar la valoración del producto respecto a la competencia.",
    ]

    # índices con valores erróneos en sentimiento
    idx_erroneos = df.index[df["sentimiento"].isin(valores_erroneos)]

    # restaurar sentimiento y producto desde df_productos
    df.loc[idx_erroneos, ["sentimiento", "producto"]] = df_productos.loc[idx_erroneos, ["sentimiento", "producto"]]

    return df, idx_erroneos


In [ ]:
df, indices_corregidos = restaurar_sentimiento_y_producto(df, df_productos)

print(f"Se han restaurado {len(indices_corregidos)} filas en 'sentimiento' y 'producto'.")

In [ ]:
df.sentimiento.value_counts()

In [ ]:

df.at[313, "sentimiento"] = "positivo"
df.at[356, "sentimiento"] = "positivo"
df.at[356, "producto"] = "1: Zumo de naranja recién exprimido\n2: huevos duros"
df.at[383, "sentimiento"] = "positivo"
df.at[402, "sentimiento"] = "positivo"
df.at[690, "sentimiento"] = "negativo"
df.at[980, "sentimiento"] = "positivo"
df.at[1206, "sentimiento"] = "positivo"
df.at[1206, "producto"] = "1: ensalada Texas"



OBJETIVO='positivo\nEnsaladas Texas del mercadona'
for i in df['tweets'][df.sentimiento==OBJETIVO]:
     print(i)
df[df.sentimiento==OBJETIVO]

In [ ]:
df.to_parquet('./data/Silver/Cleaned_data_features_productos_2_clean.parquet' ,index=False) #intermedio

vemos que previamente sentimiento se había definido los sentimientos (7 que faltan) 
negativo                                                                           166433
positivo                                                                           149417
neutral                                                                             38982

y tras pasar este ultimo procesamiento y postprocesado vemos que las comparativas del los productos han aumentado considerablemente. 
negativo    160002
positivo    154141
neutral      40696

Con 6431 valoraciones negativas menos.

In [ ]:
## CATEGORIZACIÓN DE LOS PRODUCTOS MERCADONA

In [ ]:
#chunk archivo json con todo los productos de mercadona.

def generar_chunks_para_vectorizar(ruta_archivo: str) -> List[str]:
    """
    Carga el JSON y crea una lista de strings (chunks) para ser vectorizados,
    combinando categoría, subcategoría y el nombre del producto.
    """
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            datos: List[Dict] = json.load(f)

        chunks: List[str] = []
        for categoria_principal in datos:
            nombre_categoria = categoria_principal['name']

            for subcategoria in categoria_principal.get('subcategories', []):
                nombre_subcategoria = subcategoria['name']

                for producto in subcategoria.get('products', []):
                    chunks.append(f"{nombre_categoria} : {nombre_subcategoria} : {producto}")

        return chunks

    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return []

chunks_a_vectorizar = generar_chunks_para_vectorizar('./data/Silver/mercadona_data.json')
if not chunks_a_vectorizar:
    print("No se pudieron generar los chunks. Saliendo...")
    exit()

print(f"Se han generado {len(chunks_a_vectorizar)} chunks para vectorizar.")

In [ ]:
#carga el modelo para busqueda semantica en 4 bits de quiantizacion para ahorrar espacio
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)



model_name_embedding = 'intfloat/multilingual-e5-large-instruct' 
tokenizer_embedding = AutoTokenizer.from_pretrained(model_name_embedding)
model_embedding = AutoModel.from_pretrained(model_name_embedding,
                                            torch_dtype=torch.bfloat16,
                                            quantization_config=bnb_config,
                                           )


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


model_embedding.to(device)


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


batch_size = 64
dataloader = DataLoader(chunks_a_vectorizar, batch_size=batch_size, shuffle=False)

embeddings_array = []


with torch.no_grad():
    for batch_texts in tqdm(dataloader, desc="Vectorizando batches"):
        batch_dict = tokenizer_embedding(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(device)

        outputs = model_embedding(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings_normalized = F.normalize(embeddings, p=2, dim=1)

        embeddings_array.append(embeddings_normalized.cpu())

embeddings_tensor = torch.cat(embeddings_array, dim=0)

embeddings_tensor.shape

In [ ]:
##función de busqueda en los chunks

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery:{query}'

def buscar_entradas_relevantes(
    producto: str,
    embeddings_db: torch.Tensor,
    chunks: List[str],
    top_k: int = 5
) -> List[str]:
    """
    Busca las entradas más similares al producto en la base de datos de embeddings.

    Args:
        producto (str): El nombre del producto a buscar.
        embeddings_db (torch.Tensor): El tensor con los embeddings de la base de datos.
        chunks (List[str]): La lista original de textos que se vectorizaron.
        top_k (int): El número de resultados más relevantes a devolver.

    Returns:
        List[str]: Las k entradas más relevantes.
    """
    # 1. Prepara la consulta con la instrucción
    task = 'Given a product, retrieve relevant categories from a grocery store' #lo hace mejor en inglés




    consulta_con_instruct = get_detailed_instruct(task, producto)

    # 2. Vectoriza la consulta
    with torch.no_grad():
        batch_dict = tokenizer_embedding(
            [consulta_con_instruct],
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(device)

        outputs = model_embedding(**batch_dict)
        embedding_producto = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embedding_producto_normalized = F.normalize(embedding_producto, p=2, dim=1).cpu()
        del batch_dict, outputs, embedding_producto  # libera referencias
        torch.cuda.empty_cache()  # limpia la memoria GPU no utilizada

    # 3. Calcula la similitud del coseno entre la consulta y la base de datos
    puntuaciones_similitud = torch.matmul(embedding_producto_normalized, embeddings_db.T)[0]

    # 4. Obtiene los índices de las top_k entradas más similares
    indices_top_k = torch.topk(puntuaciones_similitud, k=top_k).indices.tolist()

    # 5. Devuelve los chunks originales correspondientes a esos índices
    entradas_encontradas = [chunks[i] for i in indices_top_k]

    return entradas_encontradas


def buscar_en_prompt(lista):
    resultados = []
    try:
        for i, item in enumerate(lista.split("1:")[1].split(":")):
            resultados.append(f"**{item}**") #estos son los resultados para : 
            a = buscar_entradas_relevantes(producto=item, embeddings_db=embeddings_tensor, chunks=chunks_a_vectorizar, top_k = 5)
            #resultados.append(a)
            #resultados.append("Indeterminado : Indeterminado : Productos no listados")
            categorias = [":".join(elem.split(":")[:2]).strip() for elem in a]
            resultados.append(categorias)

        
    except:
        resultados = "Indeterminado : Indeterminado : Productos no listados"
    return resultados


def formatear_lista(lista_ref):
    salida = []
    for elem in lista_ref:
        if isinstance(elem, list):
            for sub in elem:
                salida.append(sub)
        else:
            salida.append(elem)
    return "\n".join(salida)


In [ ]:
###Clasificar el producto mediante la bsuiqeda semantica, el prodcuto y el contexto

def clasificar_categoria(producto, documento):
    prompt_categoria = [{
        "role": "user",
        "content": (
        
    f"""Eres un clasificador de productos.
        Responde **solo** con la categoría y subcategoría exactas de la lista dada.
        No añadas introducciones, explicaciones, encabezados, ni texto adicional.
        No uses formato como negritas, cursivas o cualquier tipo de markdown.

        Tu tarea es clasificar el/los siguientes producto(s) indicados en 'Objeto',
        basándote en la lista de referencia y usando también la información de 'CONTEXTO'.

        ---
        **Instrucciones:**

        1.  Lee el 'Objeto' para identificar el producto y el 'CONTEXTO' para obtener pistas adicionales.
        2.  **PRIORIDAD DEL CONTEXTO:** El **CONTEXTO** tiene prioridad absoluta sobre el nombre del producto o la lista de referencia. Si el contexto contiene información clave (ej. "en farmacia", "supermercado", "marca X"), úsala como la pista principal para la clasificación, incluso si otros productos de la lista parecen similares por el nombre.
        3.  Usa la información del contexto para corregir errores o ambigüedades en el nombre del producto.
        4.  **Encuentra la coincidencia más cercana:** De toda la lista de referencia, busca la línea que contenga el nombre del producto o su variante más parecida. Esta será tu clasificación principal.
        5.  Busca la categoría y subcategoría que mejor se ajusten al producto identificado en la lista de referencia.
            -La lista de referencia sigue el siguiente formato CATEGORIAS : SUBCATEGORIAS

        6.  Si el producto podría encajar en varias categorías, elige **según el CONTEXTO**.
        7.  Si el producto no aparece o no encaja en ninguna categoría, responde con: Indeterminado : Indeterminado.
        8.  Si se están comparando dos productos entre si, asume que son de la misma categoría y asignalos al más claro.
        9.  Devuelve la respuesta en este formato, manteniendo el orden original de aparición, una categoría y subcategoría por producto:
            1: Categoria_1 : Subcategoria_1
            2: Categoria_2 : Subcategoria_2
            ...
        10.  No incluyas comillas, texto adicional ni inventes categorías.
        11. Devuelve Categoria : Subcategoría de manera completa, sin añadir producto.
        12. Solo responde usando las categorías y subcategorías de la lista de referencia.
        13. No dupliques categoría por objeto. 
        14. Si ninguna coincide, responde exactamente: Indeterminado : Indeterminado.

        Ejemplo 1:
        Objeto:
        '1: vaselina'
        
        CONTEXTO:
        'Bephantol 8,95 en farmacia, vaselina 2 en mercadona, yo no me la juego'
        
        Lista de referencia (Categoría : Subcategoría : Producto):
        'estos son los resultados para :  vaselina\nMaquillaje : Labios : Vaselina hidratante Deliplus\nFitoterapia y parafarmacia : Parafarmacia : Vaselina hidratante Deliplus\nCuidado facial y corporal : Higiene íntima : Compresa con alas noche Ausonia\nCuidado facial y corporal : Higiene íntima : Compresa con alas normal Ausonia\nCuidado facial y corporal : Perfume y colonia : Agua de colonia Deliplus Floral Infusion\nCuidado facial y corporal : Higiene íntima : Toallitas íntimas Deliplus monodosis\nCuidado facial y corporal : Gel y jabón de manos : Esponja de baño rizo suave Deliplus\nMaquillaje : Labios : Vaselina perfumada para labios Deliplus frambuesa\nCuidado facial y corporal : Gel y jabón de manos : Esponja de baño flor Deliplus exfoliación suave\nCuidado facial y corporal : Gel y jabón de manos : Esponja de baño suave Deliplus\nIndeterminado : Indeterminado : Productos no listados'        
        
        Respuesta:
        1: Fitoterapia y parafarmacia : Parafarmacia

        Ejemplo 2:
        Objeto:
        '1: vino del pescaito'

        CONTEXTO:
        'fui a Mercadona y no quedaban botellas de vino del pescaito y un tal Alex Márquez cogió una caña de pescar para darme todos los vinitos que quería, gracias por tanto'
        
        Lista de referencia (Categoría : Subcategoría : Producto):
        'estos son los resultados para :  vino del pescaito\nBodega : Vino blanco : Vino blanco suave y afrutado El Pescaito\nMarisco y pescado : Pescado fresco : Pulpo cocido\nMarisco y pescado : Pescado fresco : Patas de pulpo cocido\nMarisco y pescado : Salazones y ahumados : Boquerones en vinagre Hacendado en aceite de girasol\nMarisco y pescado : Pescado fresco : Sepia limpia\nBodega : Vino blanco : Mosto tinto Casón Histórico\nMarisco y pescado : Pescado fresco : Escalopin de salmón\nMarisco y pescado : Marisco : Navajas\nMarisco y pescado : Pescado fresco : Filetes de trucha Arco iris\nMarisco y pescado : Pescado fresco : Medio salmón\nIndeterminado : Indeterminado : Productos no listados'

        Respuesta:
        'Bodega : Vino blanco'

        
        Ahora analiza el siguiente caso:
        Objeto:
        '{producto}'
        CONTEXTO:
        '{documento}'
        Lista de referencia (Categoría : Subcategoría : Producto):
        {formatear_lista(buscar_en_prompt(producto))}

        Respuesta:"""


    )}]

    # print("=================")
    # print(prompt_categoria)


    input_ids = tokenizer.apply_chat_template(prompt_categoria, return_tensors="pt", return_dict=True, add_generation_prompt=True)
    input_ids = {k: v.to('cuda:0') for k, v in input_ids.items()}
    with torch.no_grad():
        outputs = model.generate(**input_ids, max_new_tokens=100, do_sample=False, use_cache=False,)
    response_categoria = tokenizer.decode(outputs[0],skip_special_tokens=True)

    
    del input_ids, outputs
    torch.cuda.empty_cache()

    
    return response_categoria



In [ ]:
### Precasdo
#iteramos sobre el df para sacar categorias
categorias = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Clasificando categorías"):
    producto = row["producto"]
    documento = row["tweets"]

    if producto.strip() == "No aplica":
        categorias.append("No aplica")
        continue

    try:
        c = clasificar_categoria(producto, documento)
        categorias.append(c)
    except Exception as e:
        categorias.append("error")
        print(f"Error en fila {index}: {e}")

df["categoria"] = categorias

In [ ]:
df[df.sentimiento_producto=='SI']

In [ ]:
 ### se limpian los intermedios para no tener un exceso de archivos. 
# df.to_parquet( './data/Silver/Cleaned_data_features.parquet' ,index=False)
df.to_parquet('./data/Silver/Cleaned_data_features_productos_categorias.parquet' ,index=False)